In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading in and preprocessing data

In [ ]:
# Reading in data from the 50-50 readmit vs. nonreadmit dataset

df = pd.read_csv('dfd.csv')
df.head()

In [ ]:
# Dropping certain columns

# Dropping ID numbers and dates
df = df.drop(columns=['subject_id', 'hadm_id', 'admittime', 'dischtime'])

# Dropping labevents and chartevents values of less importance based on feature selection (2/3 values for each measurement)
df = df.drop(columns=['rdw_min', 'rdw_max', 'hemoglobin_min', 'hemoglobin_max', 'creatinine_median', 'creatinine_min', 
                      'hematocrit_median', 'hematocrit_min', 'tempc_median', 'tempc_max', 'resprate_median', 
                      'resprate_min', 'wbc_median', 'wbc_max', 'inr_min', 'inr_median', 'ptt_median', 'ptt_max', 
                      'lactate_median', 'lactate_max', 'sysbp_median', 'sysbp_min', 'spo2_median', 'spo2_max', 
                      'bilirubin_median', 'bilirubin_max', 'platelet_median', 'platelet_max', 'heartrate_min',
                      'heartrate_median'])
df.head()

In [ ]:
# Converting categorical features into dummy variables

df_converted = pd.get_dummies(df)
df_converted.head()

In [ ]:
# Splitting dataframe into data (predictors) vs. label (attributed to be predicted)

label_df = df_converted.pop('followed_by_readmit')
data_df = df_converted
print('label_df:\n', label_df.head(), 2*'\n', 'data_df:\n', data_df.head())

In [6]:
# Converting dataframes to NumPy arrays

label = label_df.values
data = data_df.values

In [7]:
label

array([ True,  True,  True, ..., False, False, False])

In [8]:
data

array([[70, 8, 0, ..., 0, 0, 0],
       [42, 19, 0, ..., 0, 0, 0],
       [60, 8, 0, ..., 0, 1, 0],
       ...,
       [74, 19, 0, ..., 0, 0, 0],
       [67, 7, 0, ..., 0, 0, 0],
       [59, 9, 0, ..., 0, 0, 0]], dtype=object)

## Train/test split

In [9]:
# 80/20 train-test split

from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(data, label, train_size=0.8, test_size=0.2, random_state=10)

print('Training data:', train_data.shape, '\tTest data:', test_data.shape)
print('Training labels:', train_label.shape, '\tTest labels:', test_label.shape)

Training data: (4873, 110) 	Test data: (1219, 110)
Training labels: (4873,) 	Test labels: (1219,)


In [10]:
# Cross-validation on the training set (no need to do explicitly for Logistic Reg. since has a cv parameter, but just in case...)

#from sklearn.model_selection import KFold
#kf = KFold(n_splits=5, random_state=10)
#for train, test in kf.split(train_data, train_label):
    #print(test)

Note that cross-validation will be performed with the training data for each machine learning algorithm tested below.

## Logistic regression

From http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression:
"LogisticRegressionCV implements Logistic Regression with builtin cross-validation to find out the optimal C parameter (similar to what GridSearchCV might do)."

In [11]:
from sklearn.linear_model import LogisticRegressionCV

First, let's try using the `liblinear` solver (recommended for small datasets) with 'l1' penalty:

In [12]:
# Instantiating logistic regression estimator object; cv=5 indicates 5-fold cross-validation

lrcv_l1 = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', random_state=10)

In [13]:
%%time

lrcv_l1.fit(train_data, train_label)

Wall time: 2min 52s


LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=10,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [14]:
lrcv_l1_pred = lrcv_l1.predict(test_data)
lrcv_l1_pp = lrcv_l1.predict_proba(test_data)

Running self.classes_ on the result from fitting the model to the data (see commented code below) tells us that the probability estimates for positive class ("True") are in column 1 rather than column 0 of the result from `predict_proba()` - this is what we'll need to use when calculating the ROC-AUC.

In [15]:
#lrcv_l1.fit(train_data, train_label).classes_

In [16]:
from sklearn.metrics import roc_auc_score

lrcv_l1_probs = lrcv_l1.predict_proba(test_data)[:,1]
print('ROC-AUC score with l1 penalty:', roc_auc_score(test_label, lrcv_l1_probs))

ROC-AUC score with l1 penalty: 0.7012404754976451


What if we used 'l2' penalty instead?

In [17]:
%%time
# Repeating the code above for logistic regression but with l2 penalty

lrcv_l2 = LogisticRegressionCV(cv=5, penalty='l2', solver='liblinear', random_state=10)
lrcv_l2.fit(train_data, train_label)
lrcv_l2_pred = lrcv_l2.predict(test_data)
lrcv_l2_pp = lrcv_l2.predict_proba(test_data)
lrcv_l2_probs = lrcv_l2.predict_proba(test_data)[:,1]
print('ROC-AUC score with l2 penalty:', roc_auc_score(test_label, lrcv_l2_probs))

ROC-AUC score with l2 penalty: 0.6992008578787977
Wall time: 28.6 s


Using the liblinear solver, l1 penalty has a very slightly better ROC-AUC score.

Now, let's get the confusion matrix and precision/recall/F1-score.

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_label, lrcv_l1_pred)

array([[385, 206],
       [237, 391]], dtype=int64)

In [19]:
from sklearn.metrics import classification_report

print(classification_report(test_label, lrcv_l1_pred))

             precision    recall  f1-score   support

      False       0.62      0.65      0.63       591
       True       0.65      0.62      0.64       628

avg / total       0.64      0.64      0.64      1219



## Stochastic Gradient Descent

We will test SGD optimization with logistic regression and modified Huber loss functions. From the documentation (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier), "‘modified_huber’ is another smooth loss that brings tolerance to outliers as well as probability estimates". For each, we will test 'l2', 'l1', and 'elasticnet' penalties.

According to the documentation (http://scikit-learn.org/stable/modules/sgd.html#tips-on-practical-use), "finding a reasonable regularization term $\alpha$ is best done using GridSearchCV, usually in the range 10.0**-np.arange(1,7)."

In [49]:
# Importing SGDClassifier and GridSearchCV, creating alphas and penalties dictionaries for use with GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

alphas = {'alpha': 10.0**-np.arange(1,7)}
penalties = {'l2': 0, 'l1': 1, 'elasticnet': 0.5}

### SGD with log loss

In [64]:
%%time
# Using GridSearchCV to find optimal value for alpha for each penalty type, using ROC-AUC score as with other models

gs_log_scores = {}
gs_log_param = {}

for k,v in penalties.items():
    print('\nPenalty:', k, '\tElastic Net mixing parameter:', v)
    gs_log = GridSearchCV(SGDClassifier(loss='log', penalty=k, l1_ratio=v, random_state=10), param_grid=alphas,
                          scoring='roc_auc', cv=5, verbose=3)
    gs_log_fit = gs_log.fit(train_data, train_label)
    score = gs_log.score(test_data, test_label)
    gs_log_scores[k] = score
    param = gs_log.best_params_
    gs_log_param[k] = param
    print('ROC-AUC score on test set for', k, 'penalty: %f' % score)
    print('Best alpha for', k, 'penalty: %s' % param)

print('\nFinal results:')
print(gs_log_param, gs_log_scores)


Penalty: l2 	Elastic Net mixing parameter: 0
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6226666778879333, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6408493376647423, total=   0.0s
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and defa

[CV] .............. alpha=0.1, score=0.6651335611250442, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6039518791666139, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6329711199099315, total=   0.0s
[CV] alpha=0.01 ......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] .............. alpha=0.01, score=0.606184039992594, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............. alpha=0.01, score=0.6050310548551614, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............. alpha=0.01, score=0.6656090622948613, total=   0.0s
[CV] alpha=0.01 ......................................................


...

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-05, score=0.6184235493542145, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] .............. alpha=1e-06, score=0.62964770833684, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6194728248977461, total=   0.0s
[CV] alpha=1e-06 .....................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.6608877143963239, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6347885120576168, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6332747213823986, total=   0.0s
ROC-AUC score on test set for elasticnet penalty: 0.679306
Best alpha for elasticnet penalty: {'alpha': 0.01}

Final results:
{'l2': {'alpha': 1e-05}, 'l1': {'alpha': 0.01}, 'elasticnet': {'alpha': 0.01}} {'l2': 0.6288892840591893, 'l1': 0.6381955446344855, 'elasticnet': 0.679305829480423}
Wall time: 7.62 s


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done  

In [67]:
log_penalty = max(gs_log_scores, key=lambda key: gs_log_scores[key])
print('Best performance for SGD with log loss ->', log_penalty, '\nROC-AUC score:', gs_log_scores[log_penalty],
      '\nalpha:', gs_log_param[log_penalty])

Best performance for SGD with log loss -> elasticnet 
ROC-AUC score: 0.679305829480423 
alpha: {'alpha': 0.01}


Using the best performer to get predicted values and other evaluation metrics:

In [72]:
sgd_log_best = SGDClassifier(loss='log', penalty=log_penalty, l1_ratio=penalties[log_penalty], random_state=10)
sgd_log_best.fit(train_data, train_label)
sgd_log_pred = sgd_log_best.predict(test_data)

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [75]:
sgd_log_pred

array([False, False, False, ..., False, False, False])

In [74]:
confusion_matrix(test_label, sgd_log_pred)

array([[563,  28],
       [555,  73]], dtype=int64)

In [76]:
print(classification_report(test_label, sgd_log_pred))

             precision    recall  f1-score   support

      False       0.50      0.95      0.66       591
       True       0.72      0.12      0.20       628

avg / total       0.62      0.52      0.42      1219



### SGD with modified Huber loss

In [65]:
%%time
# Using GridSearchCV to find optimal value for alpha for each penalty type, using ROC-AUC score as with other models

gs_hub_scores = {}
gs_hub_param = {}

for k,v in penalties.items():
    print('\nPenalty:', k, '\tElastic Net mixing parameter:', v)
    gs_hub = GridSearchCV(SGDClassifier(loss='modified_huber', penalty=k, l1_ratio=v, random_state=10), param_grid=alphas,
                          scoring='roc_auc', cv=5, verbose=3)
    gs_hub_fit = gs_hub.fit(train_data, train_label)
    score_hub = gs_hub.score(test_data, test_label)
    gs_hub_scores[k] = score_hub
    param_hub = gs_hub.best_params_
    gs_hub_param[k] = param_hub
    print('ROC-AUC score on test set for', k, 'penalty: %f' % score_hub)
    print('Best alpha for', k, 'penalty: %s' % param_hub)

print('\nFinal results:')
print(gs_hub_param, gs_hub_scores)


Penalty: l2 	Elastic Net mixing parameter: 0
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.1, score=0.62756055275959, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6362626449647372, total=   0.0s
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and defa

[CV] .............. alpha=0.1, score=0.6646117722307316, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6414466610163059, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6134436418683298, total=   0.0s
[CV] alpha=0.01 ......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] .............. alpha=0.01, score=0.605868441871034, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............. alpha=0.01, score=0.6315034252916127, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............. alpha=0.01, score=0.6580178754776053, total=   0.0s
[CV] alpha=0.01 ......................................................


...

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-05, score=0.6441664242071574, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6087719445893858, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6194686169227921, total=   0.0s
[CV] alpha=1e-06 .....................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.6608877143963239, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6347632119349111, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6332873714437515, total=   0.0s
ROC-AUC score on test set for elasticnet penalty: 0.629892
Best alpha for elasticnet penalty: {'alpha': 1e-05}

Final results:
{'l2': {'alpha': 0.001}, 'l1': {'alpha': 0.0001}, 'elasticnet': {'alpha': 1e-05}} {'l2': 0.6586941058553462, 'l1': 0.6532542274241003, 'elasticnet': 0.6298915796393891}
Wall time: 7.82 s


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    2.4s finished
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol wi

In [68]:
hub_penalty = max(gs_hub_scores, key=lambda key: gs_hub_scores[key])
print('Best performance for SGD with modified Huber loss ->', hub_penalty, '\nROC-AUC score:', gs_hub_scores[hub_penalty],
      '\nalpha:', gs_hub_param[hub_penalty])

Best performance for SGD with modified Huber loss -> l2 
ROC-AUC score: 0.6586941058553462 
alpha: {'alpha': 0.001}


Using the best performer to get predicted values and other evaluation metrics:

In [78]:
sgd_hub_best = SGDClassifier(loss='modified_huber', penalty=hub_penalty, l1_ratio=penalties[log_penalty], random_state=10)
sgd_hub_best.fit(train_data, train_label)
sgd_hub_pred = sgd_hub_best.predict(test_data)

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [79]:
sgd_hub_pred

array([False, False,  True, ..., False, False, False])

In [81]:
confusion_matrix(test_label, sgd_hub_pred)

array([[546,  45],
       [501, 127]], dtype=int64)

In [82]:
print(classification_report(test_label, sgd_hub_pred))

             precision    recall  f1-score   support

      False       0.52      0.92      0.67       591
       True       0.74      0.20      0.32       628

avg / total       0.63      0.55      0.49      1219

